In [1]:
import tensorflow as tf
import numpy as np
import models
import helpers
import data_utils

In [2]:
metadata, idx_q, idx_a = data_utils.ourmodel.data_util.load_data()
(trainX, trainY), (testX, testY), (validX, validY) = helpers.split_dataset(idx_q, idx_a)

In [3]:

# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 30


In [4]:
idx_q.shape

(11, 500)

In [5]:
from models import seq2seq_wrapper
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/checkpoint/',
                               emb_dim=emb_dim,
                               num_layers=3,
                             epochs = 50,
                                lr = 0.02,
                                attention = True,
                                celltype = 'GRU'
                               )

<log> Building Graph <log> Building Input variables of Graph <log> Cells <log> Building Actual model GradientDescentOptimizer.. </log>

In [ ]:
batch_size = 50

val_batch_gen = helpers.rand_batch_gen(validX, validY, 10)
train_batch_gen = helpers.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)

training started

Model saved to disk at iteration #1
val   loss : 2.378973

Model saved to disk at iteration #2
val   loss : 0.702355

Model saved to disk at iteration #3
val   loss : 0.748692

Model saved to disk at iteration #4
val   loss : 0.732520

Model saved to disk at iteration #5
val   loss : 0.740061


In [ ]:

sess = model.restore_last_session()

In [ ]:
input_,output_ = val_batch_gen.next()
output = model.predict(sess, input_)
print(output.shape)

In [ ]:
replies = []

for ii, oi,ot in zip(input_.T, output,output_.T):
    q = helpers.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = helpers.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    rs = helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' ')
    #if decoded.count('unk') == 0:
     #   if decoded not in replies:
    print('Review : [{0}]; Summary : [{1}];real summary :[{2}] '.format(q, ' '.join(decoded),' '.join(rs)))
    #print "Real Summary %s",(helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' '))
    replies.append(decoded)